## <center> PSYC 178 Computational Foundations for Neuroscience Assignment 6</center>

### <center> Era Songzhi Wu </center>

In this notebook I'll be performing three analyses on a sample dataset from the course website. This will be a Python counterpart for the Matlab analysis script of the same data (https://torwager.github.io/ComputationalFoundations/glm3_ashardataset.html).

In [2]:
# load necessary packages
import pandas as pd
import numpy as np
import math
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats
from scipy.stats import t, f
import warnings
#warnings.filterwarnings('ignore')

In [10]:
backpain_all = pd.read_csv("/Users/erawu/Dropbox (Dartmouth College)/Academics./PSYC 178 - Computational Foundations of Neuroscience./Assignment./Assignment 1_ESW./Backpain data/Ashar2022_outcomes_demographics.csv")
backpain_all.head()

,id,time,group,pain_avg,bpi_intensity,bpi_interference,odi,promis_dep,promis_anger,promis_anxiety,...,exercise,handedness,sses,married_or_living_as_marri,age,weight,gender,current_opioid_use,backpain_length,is_patient
0,12,1,3.0,2.0,2.50,1.000000,12.0,20.0,13.0,24.0,...,3.0,1.0,4.0,0.0,21.0,115.0,1.0,<undefined>,3.0,1
1,12,2,3.0,3.0,2.75,2.142857,12.0,22.0,18.0,28.0,...,3.0,1.0,4.0,0.0,21.0,115.0,1.0,<undefined>,3.0,1
2,14,1,3.0,2.0,2.50,1.142857,26.0,8.0,5.0,9.0,...,4.0,1.0,9.0,0.0,66.0,120.0,2.0,<undefined>,35.0,1
3,14,2,3.0,3.0,2.00,1.285714,20.0,10.0,5.0,12.0,...,4.0,1.0,9.0,0.0,66.0,120.0,2.0,<undefined>,35.0,1
4,15,1,3.0,2.0,2.25,1.000000,10.0,10.0,9.0,8.0,...,3.0,1.0,6.0,1.0,41.0,185.0,1.0,<undefined>,10.0,1


In [36]:
backpain_stacked = backpain_all[['id', 'time', 'group', 'bpi_intensity', 'gender', 'is_patient']]
backpain_stacked = backpain_stacked[backpain_stacked['is_patient']==1]
backpain_stacked

,id,time,group,bpi_intensity,gender,is_patient
0,12,1,3.0,2.50,1.0,1
1,12,2,3.0,2.75,1.0,1
2,14,1,3.0,2.50,2.0,1
3,14,2,3.0,2.00,2.0,1
4,15,1,3.0,2.25,1.0,1
...,...,...,...,...,...,...
282,1294,2,2.0,5.50,2.0,1
283,1302,1,3.0,1.00,2.0,1
284,1302,2,3.0,1.00,2.0,1
285,1319,1,2.0,4.50,1.0,1


In [61]:
backpain_pivot = backpain_stacked.pivot(index=['id', 'gender', 'group', 'is_patient'],  columns='time', values='bpi_intensity')
backpain_pivot.columns = ['BPI_Time1', 'BPI_Time2']
backpain_pivot.reset_index(inplace=True)
backpain_pivot['bpi_change'] = backpain_pivot['BPI_Time2'] - backpain_pivot['BPI_Time1']
backpain_pivot = backpain_pivot.dropna()
backpain_pivot

,id,gender,group,is_patient,BPI_Time1,BPI_Time2,bpi_change
0,12,1.0,3.0,1,2.50,2.75,0.25
1,14,2.0,3.0,1,2.50,2.00,-0.50
2,15,1.0,3.0,1,2.25,2.75,0.50
3,18,1.0,2.0,1,4.50,2.25,-2.25
4,23,2.0,2.0,1,2.50,2.25,-0.25
...,...,...,...,...,...,...,...
147,1268,2.0,2.0,1,4.50,2.75,-1.75
148,1277,2.0,2.0,1,4.25,5.00,0.75
149,1294,2.0,2.0,1,5.50,5.50,0.00
150,1302,2.0,3.0,1,1.00,1.00,0.00


In [62]:
#create contrasts, design matrix, observation matrix
indic = pd.DataFrame({
    'Group1': (backpain_pivot['group'] == 1).astype(float),
    'Group2': (backpain_pivot['group'] == 2).astype(float),
    'Group3': (backpain_pivot['group'] == 3).astype(float)
})
indic = pd.get_dummies(indic).values
C = np.array([[1, 1, 1], [1, -1, 0], [-0.5, -0.5, 1]]).T
X = indic @ C
y = backpain_pivot['bpi_change']

print(X.shape)
print(y)

(135, 3)
0      0.25
1     -0.50
2      0.50
3     -2.25
4     -0.25
       ... 
147   -1.75
148    0.75
149    0.00
150    0.00
151   -0.75
Name: bpi_change, Length: 135, dtype: float64


In [64]:
pd.DataFrame(X).isna().any().any()
pd.DataFrame(y).isna().any().any()

False

**1. Custom GLM**

In this section, I'll
code the core GLM equations,
create a design matrix(X) that denotes differences among the 3 groups,
estimate the model and report output,
validate by comparing results with an established GLM function,
turn my code into a function.

In [101]:
# core GLM equations

xtinv = np.linalg.inv(X.T @ X)
bhat = xtinv @ X.T @ y
e = y - X @ bhat
n, p = X.shape
var_bhat = (e.T @ e) / (n - p) * xtinv
t_stat = bhat / np.sqrt(np.diag(var_bhat))
dfe = n - p
pval = 2 * (1 - t.cdf(np.abs(t_stat), dfe))
se = np.sqrt(np.diag(var_bhat))
syy = ((y - np.mean(y)) ** 2).sum()
f_stat = ((syy - (e.T @ e)) / (p - 1)) / ((e.T @ e) / (n - p))
pval_full_model = 1 - f.cdf(f_stat, p - 1, n - p)
r_squared = 1 - ((e.T @ e) / syy)

# print results given sample data
cnames = ['Intercept' 'PRT-Pla' 'Treat-Control']
stat_table = pd.DataFrame({'bhat': bhat, 'SE': se, 't': t_stat, 'P': pval})

print('Analysis of BPI change (negative = improvement)')
print(stat_table)

# print(bhat)
# print(se)
# print(t_stat)
# print(pval)
# print(cnames)

Analysis of BPI change (negative = improvement)
       bhat        SE         t             P
0 -0.275540  0.052422 -5.256159  5.757343e-07
1 -0.618017  0.063210 -9.777265  0.000000e+00
2  0.070243  0.194530  0.361093  7.186076e-01


**2. Scaling**

In this section, I'll explore the relationship between beta-hats and group means. The end result should be that the product of estimated coefficient and the normed Gram matrix of contrasts is the same as differences in group means.

In [130]:

# calculate mean differences & group mean differences
y = backpain_pivot['bpi_change'].values
means_by_group = [np.nanmean(y[indic[:, 0] == 1]), np.nanmean(y[indic[:, 1] == 1]), np.nanmean(y[indic[:, 2] == 1])]
contra = np.array([[1, 1, 1], [1, -1, 0], [-0.5, -0.5, 1]]).T
group_mean_diff = np.dot(means_by_group, contra)

# calculate Gram matrix and norm of columns
norm_col = np.linalg.norm(contra[:, 2]) ** 2
gram = np.dot(contra.T, contra)
contra_normed = contra / np.diag(gram)

X = np.dot(indic, contra_normed)
#bhat @ np.diag(np.linalg.inv(contra.T @ contra))

In [131]:
xtinv = np.linalg.inv(X.T @ X)
bhat = xtinv @ X.T @ y
e = y - X @ bhat
n, p = X.shape
var_bhat = (e.T @ e) / (n - p) * xtinv
t_stat = bhat / np.sqrt(np.diag(var_bhat))
dfe = n - p
pval = 2 * (1 - t.cdf(np.abs(t_stat), dfe))
se = np.sqrt(np.diag(var_bhat))
syy = ((y - np.mean(y)) ** 2).sum()
f_stat = ((syy - (e.T @ e)) / (p - 1)) / ((e.T @ e) / (n - p))
pval_full_model = 1 - f.cdf(f_stat, p - 1, n - p)
r_squared = 1 - ((e.T @ e) / syy)

# print results given sample data
cnames = ['Intercept' 'PRT-Pla' 'Treat-Control']
stat_table = pd.DataFrame({'bhat': bhat, 'SE': se, 't': t_stat, 'P': pval})

print("Stats table")
print(stat_table)
print("Group mean differences")
print(group_mean_diff)

Stats table
       bhat        SE          t             P
0 -4.078457  0.321401 -12.689636  0.000000e+00
1 -1.767045  0.265260  -6.661569  6.669629e-10
2  1.265293  0.224781   5.629008  1.042251e-07
Group mean differences
[-4.07845745 -1.76704545  1.26529255]


**3. Contrasts**

In this section, I'll turn the core GLM equations into a function that allows for contrasts. 

In [188]:
def GLM_with_contrast(X, y, C, cnames):
    xtinv = np.linalg.inv(X.T @ X)
    bhat = xtinv @ X.T @ y
    e = y - X @ bhat
    n, p = X.shape
    var_bhat = (e.T @ e) / (n - p) * xtinv
    t_stat = bhat / np.sqrt(np.diag(var_bhat))
    dfe = n - p
    pval = 2 * (1 - t.cdf(np.abs(t_stat), dfe))
    se = np.sqrt(np.diag(var_bhat))
#     syy = ((y - np.mean(y)) ** 2).sum()
#     f_stat = ((syy - (e.T @ e)) / (p - 1)) / ((e.T @ e) / (n - p))
#     pval_full_model = 1 - f.cdf(f_stat, p - 1, n - p)
#     r_squared = 1 - ((e.T @ e) / syy)
    
    #add contrast-related variables
    chat = C.T @ bhat
    var_c = e.T @ e / (n - p) * C.T @ xtinv @ C
    se_c = np.sqrt(np.diag(var_c))
    t_c = chat / se_c
    pval_c = 2 * (1 - t.cdf(np.abs(t_c), dfe))
    
#     return bhat, se, t_stat, pval
#     return chat, se_c, t_c, pval_c
    
    stat_table = pd.DataFrame({
        'Name': xnames,
        'bhat': bhat,
        'SE': se,
        't': t_stat,
        'P': pval
    })
    contrast_table = pd.DataFrame({
        'Contrast': cnames,
        'bhat': chat,
        'SE': se_c,
        't': t_c,
        'P': pval_c
    })

    print('Model parameters:')
    print(stat_table)
    print('Contrasts:')
    print(contrast_table)

#     return stat_table, contrast_table

We can test this contrast-GLM function on the sample data.

In [189]:
indic = pd.DataFrame({
    'Group1': (backpain_pivot['group'] == 1).astype(float),
    'Group2': (backpain_pivot['group'] == 2).astype(float),
    'Group3': (backpain_pivot['group'] == 3).astype(float)
})
indic = pd.get_dummies(indic).values
C = np.array([[1, 1, 1], [1, -1, 0], [-0.5, -0.5, 1]]).T
X = indic @ C
y = backpain_pivot['bpi_change']
xnames = ['group 1', 'group 2', 'group 3']
cnames = np.array(['Intercept', 'PRT-Pla', 'Treat-Control'])

GLM_with_contrast(X, y, C, cnames)


Model parameters:
      Name      bhat        SE          t             P
0  group 1 -1.359486  0.107134 -12.689636  0.000000e+00
1  group 2 -0.883523  0.132630  -6.661569  6.669629e-10
2  group 3  0.843528  0.149854   5.629008  1.042251e-07
Contrasts:
        Contrast      bhat        SE          t             P
0      Intercept -1.399480  0.224781  -6.225979  5.920007e-09
1        PRT-Pla -0.475963  0.170494  -2.791668  6.022610e-03
2  Treat-Control  1.965033  0.173846  11.303326  0.000000e+00
